### Vectors

This notebook contains example code from *Fluent Python*, by Luciano Ramalho.

Code by Luciano Ramalho, modified by Allen Downey.

MIT License: https://opensource.org/licenses/MIT

In [ ]:
from math import hypot

class Vector:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [ ]:
beer_card = Card('7', 'diamonds')
beer_card

You can access the fields of a card by name.

In [ ]:
beer_card.rank, beer_card.suit

Or by index.

In [ ]:
beer_card[0], beer_card[1]

A drawback of using namedtuples is that you can't define methods for them in the usual way.

But you can [monkey-patch](https://en.wikipedia.org/wiki/Monkey_patch) them by defining a function and then making it an attribute of `Card`.  For example, here's a function that generates a string representation of a card:

In [ ]:
def card_to_str(card):
    return '%s of %s' % card

card_to_str(beer_card)

Here's how we can make that function behave like a method.  When we pass a card to `print`, Python invokes the special method `__str__`

In [ ]:
Card.__str__ = card_to_str
print(beer_card)

Now let's instantiate a `FrenchDeck`.

When we call `len`, Python invokes the `__len__` method on the deck. 

In [ ]:
deck = FrenchDeck()
len(deck)

When we use the bracket operator, Python invokes the `__getitem__` method:

In [ ]:
deck[3]

And that means that the slice operator works, too:

In [ ]:
deck[:3]

Aside: In this context, we don't get the string generated by `__str__`; we get the one generated by `__repr__` (read about that [here](https://docs.python.org/3/reference/datamodel.html#basic-customization)) 



**Exercise** Use a slice to select the last three cards in the deck.

In [ ]:
deck[-3:]

Because `FrenchDeck` provides `__len__` and `__getitem__`, it is considered a sequence, which means that the `in` operator works:

In [ ]:
Card('Q', 'hearts') in deck

**Exercise** Make up a card that doesn't exist and confirm that `in` yields `False`.

In [ ]:
# Solution

Card('Z', 'clubs') in deck

Because a `FrenchDeck` is considered a sequence, it works in a for loop.

In [ ]:
for card in deck:
    print(card)

Other methods that work with sequences, like `random.choice`, will work with decks:

In [ ]:
from random import choice
choice(deck)

Sadly, `shuffle` doesn't work because we haven't provided `__setitem__`, so a deck is an immutable sequence:

In [ ]:
from random import shuffle

# This should raise a TypeError
shuffle(deck)

We can use `sorted` to iterate through the cards in the order determined by tuple comparison:

In [ ]:
for card in sorted(deck):
    print(card)

If we want an ordering that makes more sense for cards, we can define a function that maps from a card to an integer:

In [ ]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high_ordering(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [ ]:
spades_high_ordering(Card('2', 'clubs'))

In [ ]:
spades_high_ordering(Card('A', 'spades'))

And then pass this funcition as a key to `sorted`:

In [ ]:
for card in sorted(deck, key=spades_high_ordering):
    print(card)

**Exercise**  Define a new ordering that sorts the cards by suit first and then by rank, so all clubs come first, followed by all diamonds, etc.

In [ ]:
# Solution

def spades_high_ordering_suit_first(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return suit_values[card.suit] * len(FrenchDeck.ranks) + rank_value

for card in sorted(deck, key=spades_high_ordering_suit_first):
    print(card)

**Exercise**  Write a method called `setcard` that takes a deck, an index, and a card, and assigns the card to the deck at the given position.  Then monkey-patch `FrenchDeck` to provide `__setitem__` as a method.  Test it by assigning a new card like this:

    deck[0] = Card('A', 'spades')

Then shuffle the deck using `random.shuffle`.

In [ ]:
# Solution

def setcard(deck, position, card):
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = setcard

deck[0] = Card('A', 'spades')

from random import shuffle

shuffle(deck)
for card in deck:
    print(card)

We should have two Aces of spades now, which we can confirm by checking the number of unique cards:

In [ ]:
len(set(deck))